In [1]:
import findspark
findspark.init('/home/siddharth/spark-2.4.1-bin-hadoop2.7/')

from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
data = spark.read.csv('./Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Linear_Regression/Ecommerce_Customers.csv',
                      inferSchema = True,header = True)

In [5]:
data.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)]

In [6]:
data.head(1)[0].asDict()

{'Email': 'mstephenson@fernandez.com',
 'Address': '835 Frank TunnelWrightmouth, MI 82180-9605',
 'Avatar': 'Violet',
 'Avg Session Length': 34.49726772511229,
 'Time on App': 12.65565114916675,
 'Time on Website': 39.57766801952616,
 'Length of Membership': 4.0826206329529615,
 'Yearly Amount Spent': 587.9510539684005}

In [7]:
# Formatting the data for ML algos

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [8]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [9]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [10]:
assembler = VectorAssembler(inputCols=['Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent'], outputCol = 'features')

In [11]:
output_data = assembler.transform(data)

In [12]:
output_data

DataFrame[Email: string, Address: string, Avatar: string, Avg Session Length: double, Time on App: double, Time on Website: double, Length of Membership: double, Yearly Amount Spent: double, features: vector]

In [13]:
output_data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

In [14]:
final_df = output_data.select('features','Yearly Amount Spent')
final_df.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [15]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [16]:
train_data,test_data = final_df.randomSplit([0.7,0.3])

In [17]:
lr_model = lr.fit(train_data)

In [18]:
lr_model.coefficients

DenseVector([-0.0, -0.0, -0.0, -0.0, 1.0])

In [19]:
lr_model.explainParams()

'aggregationDepth: suggested depth for treeAggregate (>= 2) (default: 2)\nelasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty (default: 0.0)\nepsilon: The shape parameter to control the amount of robustness. Must be > 1.0. (default: 1.35)\nfeaturesCol: features column name (default: features)\nfitIntercept: whether to fit an intercept term (default: True)\nlabelCol: label column name (default: label, current: Yearly Amount Spent)\nloss: The loss function to be optimized. Supported options: squaredError, huber. (Default squaredError) (default: squaredError)\nmaxIter: maximum number of iterations (>= 0) (default: 100)\npredictionCol: prediction column name (default: prediction)\nregParam: regularization parameter (>= 0) (default: 0.0)\nsolver: The solver algorithm for optimization. Supported options: auto, normal, l-bfgs. (Default auto) (default: auto)\nstandardization: whether to standardize t

In [20]:
summ = lr_model.evaluate(test_data)

In [21]:
summ.r2

1.0

In [22]:
unlabel_data = test_data.select('features')

In [23]:
predictions = lr_model.transform(unlabel_data)

In [24]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...|319.92886980319776|
|[30.8162006488763...|266.08634094846826|
|[30.8364326747734...| 467.5019004269962|
|[30.8794843441274...| 490.2065999848581|
|[31.0472221394875...|392.49739918902395|
|[31.0662181616375...|  448.933293207675|
|[31.1280900496166...| 557.2526867470542|
|[31.3584771924370...|495.17595044948246|
|[31.4252268808548...| 530.7667186547628|
|[31.5147378578019...|489.81248799645493|
|[31.5261978982398...|409.09452619233355|
|[31.5316044825729...| 436.5156057293627|
|[31.5702008293202...| 545.9454921414022|
|[31.6005122003032...|479.17285149110916|
|[31.6610498227460...|416.35835357990567|
|[31.7216523605090...| 347.7769266318767|
|[31.7366356860502...| 496.9334462555404|
|[31.8093003166791...| 536.7718993628416|
|[31.8279790554652...|440.00274754693936|
|[31.8530748017465...|459.28512346235317|
+--------------------+------------

In [25]:
summ.rootMeanSquaredError

5.827091838792482e-12